In [54]:
#dependencies

from Bio import SeqIO
import pandas as pd


from Bio import Entrez
import statistics
import re
import collections
import ete3
import itertools



import os

In [52]:
#from zenodo
foldseek_f = '/flaviviridae_foldseek_output/Flaviviridae_Foldseek_Results.tsv'


In [53]:
def combalign(combrefal, reftoal, outfile):

    fasta = ''

    for rec in SeqIO.parse(combrefal, 'fasta'):

        thishseq = str(rec.seq)
        #find where the hit's reference match fits into the full ref protein
        thematch = re.search(thishseq.replace('-', ''), pdbparse('ref_structures/%s'%rec.id))

        dashes = [m.start() for m in re.finditer("-", thishseq)]

        # print(alpos)
        print(thematch)

        for alrec in SeqIO.parse(reftoal[rec.id], 'fasta'):
            if alrec.id.split('|')[0] not in fasta:
                theseq = str(alrec.seq)
                #keep only the matching bit of the reference alignment 
                theseq = theseq[thematch.start():thematch.end()]

                for d in dashes:
                    theseq = theseq[:d] + '-' +  theseq[d:]
                if set(theseq) != {'-'}:
                    fasta = fasta + '>%s\n%s\n'%(alrec.id, theseq)


    with open(outfile, 'w') as out:
        out.write(fasta[:-1])

# Use foldseek reference alignments | final foldseek search 

The foldseek results dataframe used here (`Foldseek_020424.tsv`) was produced with: `foldseek easy-search Mifsud_2024_Data/structures Mifsud_2024_Data/reference_structures Foldseek_020424.tsv tmp --format-output "query,target,alnlen,fident,qaln,taln,evalue,bits,prob,lddt,alntmscore" --exhaustive-search -e 0.1`

In [4]:
foldsall = pd.read_csv(foldseek_f, sep='\t')
foldsall.columns = ['query', 'hit', 'len', 'fident', 'qseq', 'hseq', 'eval', 'bits', 'prob', 'lddt', 'alntmscore']

In [5]:
foldsall

,query,hit,len,fident,qseq,hseq,eval,bits,prob,lddt,alntmscore
0,HPHV_Hepatitis_C_virus_genotype_1_block_4_Alph...,HPHV_E2_HCV_NC_004102.pdb,298,1.000,LLTPGAKQNIQLINTNGSWHINSTALNCNESLNTGWLAGLFYQHKF...,LLTPGAKQNIQLINTNGSWHINSTALNCNESLNTGWLAGLFYQHKF...,5.020000e-60,2134,1.000,0.869,0.951
1,HPPV_Bat_pegivirus_PDB-303_block_2_AlphaFold.pdb,HPPV_E2_BPV_KC796073.pdb,280,1.000,RFPGFDECDCTGRGFTRSNNTQVCICPFGSLVWQGREARQKLSWAQ...,RFPGFDECDCTGRGFTRSNNTQVCICPFGSLVWQGREARQKLSWAQ...,2.670000e-59,2006,1.000,0.835,0.922
2,HPPV_Hepatitis_GB_virus_A_block_3_AlphaFold.pdb,HPPV_E2_GBV-A_NC_001837.pdb,294,1.000,GNGTVVCECDFGKMHWAPKLCSGLLWRDGAREGTVWTLPRVCPREV...,GNGTVVCECDFGKMHWAPKLCSGLLWRDGAREGTVWTLPRVCPREV...,2.390000e-57,2047,1.000,0.854,0.919
3,PLPV_Linda_virus_strain_Austria1_block_7_Alpha...,PLPV_E2_LPV_NC_035432.pdb,296,1.000,LECNFELQYALAGNTSMSLLGPTALKTQWYQAADGVKITDGVVTVI...,LECNFELQYALAGNTSMSLLGPTALKTQWYQAADGVKITDGVVTVI...,1.240000e-56,2382,1.000,0.948,0.957
4,PLPV_Pestivirus_giraffe-1_H138_block_7_AlphaFo...,PLPV_E2_GPV_NC_003678.pdb,300,1.000,ARSKRIGPLGAEDLVTTWHDYKFDLKIQDPLVMVYCKNDQFFVGKR...,ARSKRIGPLGAEDLVTTWHDYKFDLKIQDPLVMVYCKNDQFFVGKR...,1.030000e-55,2400,1.000,0.978,0.977
...,...,...,...,...,...,...,...,...,...,...,...
78683,HPHV_Xiamen_sepia_Stingray_hepacivirus_block_7...,HPHV_E2_BHV_NC_031947.pdb,163,0.171,PGHTLERCFYGRQLNPHINATI-----LGPNPSH----AIRTGGV-...,PRATFTLCGQGPWISPTALIKYPMAHVHWPQVAEYGEYTIRFSSSL...,1.000000e-01,36,0.150,0.401,0.293
78684,HPHV_Xiamen_sepia_Stingray_hepacivirus_block_7...,HPHV_E2_HVF_NC_038427.pdb,138,0.224,NPHINATI--LGPNPS---HAIRTG-GVLVAVNCH------TTPPF...,TPVAMLDVPGQYWGPQWRRSSIILSHGFQIGYWFNEELLGELAPSH...,1.000000e-01,30,0.090,0.358,0.223
78685,PLPV_Pangolin_pestivirus_1_isolate_ZJ-F1_block...,FJTB_NS5_MTase_7V1E.pdb,186,0.129,LDSI-VMRLSKMQ---------RGEFKQV---------------TL...,LGDLWKRRLNNCTKEEFFAYRRTGILETERDKARELLRKGETNMGL...,1.000000e-01,26,0.063,0.299,0.290
78686,HPPV_Bat_pegivirus_PDB-694_block_3_AlphaFold.pdb,PLPV_E2_PAPeV_NC_024018.pdb,202,0.118,ACGVGQQ-LTTFLTAIPATLADPVSNQPWIGRKPENPGVKGPILDF...,RLYDTRPCKEKAFTFSPA------------GE--------------...,1.000000e-01,30,0.090,0.386,0.295


The function below goes through the foldseek results of each query poyprotein matching a set of glycoprotein references (E1, E2, E), chooses the best reference protein for any of the polyprotein blocks (by bitscore) and uses that reference to get the start and end points of the corresponding glycoprotein within the polyprotein blocks. 

In [9]:
def combine_blocks(foldseekdf, aafasf, protnams, prot, ecutoff, outf):
    
    #load full aa query seqs fasta
    aaseqdic = SeqIO.to_dict(SeqIO.parse(aafasf, 'fasta'))
    #replace unwanted characters to match foldseek rez
    aaseqdic = {k.replace('/', '_'):v for k,v in aaseqdic.items()}

    #eval cutoff and sort by bits on full df
    foldsalleval = foldsall[foldsall['eval'] <ecutoff]
    foldsalleval.sort_values(by='bits', inplace=True, ascending = False) 

    fas = ''

    #for each unique query sequence sorted alphabetically
    for q in sorted(list(set([x.split('_block_')[0] for x in list(foldsall['query']) if 'block' in x]))):
        
#         print(q)

        #filter by each query virus
        qhitsdf = foldsalleval[(foldsalleval['query'].str.contains(q + '_block_'))]

        #filter by hit options (E1, E2, E)
        qhitsdf = qhitsdf[qhitsdf.hit.str.contains(protnams)]

        if len(qhitsdf) > 0:

            #make a blocknam column with the query's block number (for overlaps later)
            qhitsdf['blocknam'] = [int(x.split('_block_')[1].split('_')[0]) for x in list(qhitsdf['query'])]

            #get the best hit
            tophit = list(qhitsdf.hit)[0]
            #and only keep rows matching to that
            qhitsdf = qhitsdf[qhitsdf.hit == tophit]

            #because we have both AF and ESMfold results in here I'm gonna drop duplicate blocknams and keep the one with highest bitscore
            qhitsdf = qhitsdf.loc[qhitsdf.groupby('blocknam')['bits'].idxmax()]

            #sort by blocknam to get start and end of full match
            qhitsdf.sort_values(by='blocknam', inplace=True)
            
#             print(qhitsdf)

            if q in list(aaseqdic):

                #get query's full aa sequence from fasta file dic
                fullqseq = str(aaseqdic[q].seq)
                #need to remove Xs
                fullqseq = fullqseq.replace('X', '')

                stqseq = list(qhitsdf.qseq)[0]
                endqseq = list(qhitsdf.qseq)[-1]

    #             print(endqseq)
                #find where the query seq alignment fits into the full query protein
                stmatch = re.search(stqseq.replace('-', ''), fullqseq).start()
                endmatch = re.search(endqseq.replace('-', ''), fullqseq).end()

                #crop the protein match start to end
                fullseqmatch = fullqseq[stmatch:endmatch]

                head = '%s|%s|%i-%i'%(q, prot, stmatch+1, endmatch)

                fas = fas + '>%s\n%s\n'%(head, fullseqmatch)
            
            else:
                print(q)

    with open(outf, 'w') as out:
        out.write(fas[:-1])
    

In [24]:
combine_blocks(foldsall, '/sequences/full_genome_sequences/flaviviridae_protein_seqs.fasta', 'E1_REF|Envelope_1_|_E1_', 'E1', 0.001, 'Foldseek_020424_E1.fas')

C:\Users\spyro\AppData\Local\Temp\ipykernel_28120\546747922.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foldsalleval.sort_values(by='bits', inplace=True, ascending = False)


In [25]:
combine_blocks(foldsall, '/sequences/full_genome_sequences/flaviviridae_protein_seqs.fasta', 'E2_REF|Envelope_2_|_E2_', 'E2', 0.001, 'Foldseek_020424_E2.fas')

C:\Users\spyro\AppData\Local\Temp\ipykernel_28120\546747922.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foldsalleval.sort_values(by='bits', inplace=True, ascending = False)


In [26]:
combine_blocks(foldsall, '/sequences/full_genome_sequences/flaviviridae_protein_seqs.fasta', 'E_REF|Envelope_|_E_', 'E', 0.001, 'Foldseek_020424_E.fas')

C:\Users\spyro\AppData\Local\Temp\ipykernel_28120\546747922.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foldsalleval.sort_values(by='bits', inplace=True, ascending = False)


PLLG_Flaviviridae_sp._isolate_TIGMIC_15
PLLG_Flaviviridae_sp._isolate_TIGMIC_4
PLUN_flavi_lake_water_ORF238_SRR7883614_orf_1


Take the resulting glycoprotein sequences  and fold them with ESMfold and ColabFold and get the best structure for each sequence

<br> 

Recompiled famsa3di by changing the substitution matrix to the foldseek one as described in https://github.com/nmatzke/3diphy/blob/111d33bb30b79eb213dc129a7cdb7b7c6cdf9d5a/3DI_substmat/put_3di_into_FAMSA.R

<br>

Convert all best structures to 3di characters:

`for f in refolded_fullglyco/*.pdb; do foldseek structureto3didescriptor $f 3di/${f%.*}.dump; done`

put in single file

`cd 3di/`

`cat *.dump > refolded_fullglyco.dumpall`

`rm *.dump *.dbtype`

convert to fasta:

`awk -F'\t' '{print ">"$1"\n"$3; next}{print}' refolded_fullglyco.dumpall > refolded_fullglyco_3di.fas`

`awk -F'\t' '{print ">"$1"\n"$2; next}{print}' refolded_fullglyco.dumpall > refolded_fullglyco_aa.fas`


Separate by glycoprotein:

In [4]:
for g in ['E', 'E1', 'E2']:
    
    fasaa = '\n'.join(['>%s\n%s'%(rec.id, str(rec.seq)) for rec in SeqIO.parse('3di/refolded_fullglyco_aa.fas', 'fasta') if rec.id.split('_')[-3] == g])
    fas3di = '\n'.join(['>%s\n%s'%(rec.id, str(rec.seq)) for rec in SeqIO.parse('3di/refolded_fullglyco_3di.fas', 'fasta') if rec.id.split('_')[-3] == g])
    
    with open('3di/refolded_fullglyco_%s_aa.fas'%g, 'w') as out:
        out.write(fasaa)
    with open('3di/refolded_fullglyco_%s_3di.fas'%g, 'w') as out:
        out.write(fas3di)
    

In [45]:
allnams = list(SeqIO.to_dict(SeqIO.parse('3di/refolded_fullglyco_aa.fas', 'fasta')))

allnams = [x.split('_E_')[0] for x in allnams]
allnams = [x.split('_E1_')[0] for x in allnams]
allnams = [x.split('_E2_')[0] for x in allnams]

allnams  = sorted(list(set(allnams)))

with open('refolded_fullglyco/refolded_fullglyco_names.txt', 'w') as out:
    out.write('\n'.join(allnams))

align with `famsa3di`:

`famsa3di refolded_fullglyco_E_3di.fas refolded_fullglyco_E_3di_famsa.fas`


<br>

Now map AAs to 3di alignment and combine into AA+3di fasta file:

In [19]:
def aa_3di_convert(aafile, al3di, alout):
    
    unalaa_dic = SeqIO.to_dict(SeqIO.parse(aafile, 'fasta'))

    aaalseq = ''

    for rec in SeqIO.parse(al3di, 'fasta'):
        c=0
        diseq = str(rec.seq)
        alaa = ''
        for char in diseq:
            if char == '-':
                alaa = alaa + '-'
            else:
                alaa = alaa + str(unalaa_dic[rec.id].seq)[c]
                c = c+1

        aaalseq = aaalseq + '>%s\n%s\n'%(rec.id, alaa)

    with open(alout, 'w') as out:
        out.write(aaalseq[:-1])


In [13]:
commonpath = '3di/refolded_fullglyco_'

In [21]:

for g in ['E', 'E1', 'E2']:

    aa_3di_convert(commonpath + g + '_aa.fas',
                  commonpath + g + '_3di_famsa.fas',
                  commonpath + g + '_AA_famsa.fas')

Use iqtree ModelFinder to get the best model for each alignment:

`for f in E1 E2 E; do iqtree2 -nt 24 -s refolded_fullglyco_${f}_AA_famsa.fas -m
 TESTONLY -mset Blosum62,Dayhoff,DCMut,JTT,JTTDCMut,LG,Poisson,Poisson+FQ,Poisson,PMB,WAG,EX2,EX3,EHO,EX_EHO,3DI -mfreq FU,F -mrate E,G,R; done`
 


In [18]:
def aa_3di_comb(al3di, alaa, alout, modelpath):

    al3di_dic = SeqIO.to_dict(SeqIO.parse(al3di, 'fasta'))
    alaa_dic = SeqIO.to_dict(SeqIO.parse(alaa, 'fasta'))
    
    #combine the alignments
    combfas =  '\n'.join([ '>%s\n%s%s'%(x, str(al3di_dic[x].seq), str(alaa_dic[x].seq)) for x in list(alaa_dic)])
    
    with open(alout, 'w') as out:
         out.write(combfas)
            
    #read modelfinder output
    model3di = ''
    with open(modelpath + al3di.split('/')[-1] + '.iqtree', 'r', encoding="utf-8") as iqtf:
        model3di = [l.replace('Best-fit model according to BIC: ', '') for l in iqtf.readlines() if 'Best-fit model according to BIC: ' in l][0]
    modelaa = ''
    with open(modelpath + alaa.split('/')[-1] + '.iqtree', 'r', encoding="utf-8") as iqtf:
        modelaa = [l.replace('Best-fit model according to BIC: ', '') for l in iqtf.readlines() if 'Best-fit model according to BIC: ' in l][0]
    
    print('alignment %s has model %s'%(al3di.split('/')[-1], model3di))
    print('alignment %s has model %s'%(alaa.split('/')[-1], modelaa))
    
    #make iqtree partition nexus file
    len3di = len(al3di_dic[list(al3di_dic)[0]].seq)
    lenaa = len(alaa_dic[list(alaa_dic)[0]].seq)
    nxsstr = "#nexus\nbegin sets;\n\tcharset part1 = 1-%i;\n\tcharset part2 = %i-%i;\n\tcharpartition mine = %s:part1, %s:part2;\nend;"%(len3di, len3di+1, len3di+lenaa, model3di, modelaa)
    with open(alout.replace('.fas', '_parts.nex'), 'w') as out:
        out.write(nxsstr)
        
    print('Run as: \n\tiqtree2 -nt 22 -s %s -p %s -B 1000'%(alout.split('/')[-1], alout.split('/')[-1].replace('.fas', '_parts.nex')))

<br>

Full combined alignments:

In [19]:
aa_3di_comb(commonpath + 'E1_3di_famsa.fas', commonpath + 'E1_AA_famsa.fas', commonpath + 'E1_3di_AA_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E1_3di_famsa.fas has model 3DI+R5

alignment refolded_fullglyco_E1_AA_famsa.fas has model EX_EHO+FU+R7

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E1_3di_AA_famsa.fas -p refolded_fullglyco_E1_3di_AA_famsa_parts.nex -B 1000


In [20]:
aa_3di_comb(commonpath + 'E2_3di_famsa.fas', commonpath + 'E2_AA_famsa.fas', commonpath + 'E2_3di_AA_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E2_3di_famsa.fas has model 3DI+F+R6

alignment refolded_fullglyco_E2_AA_famsa.fas has model WAG+F+R6

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E2_3di_AA_famsa.fas -p refolded_fullglyco_E2_3di_AA_famsa_parts.nex -B 1000


In [21]:
aa_3di_comb(commonpath + 'E_3di_famsa.fas', commonpath + 'E_AA_famsa.fas', commonpath + 'E_3di_AA_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E_3di_famsa.fas has model 3DI+F+R6

alignment refolded_fullglyco_E_AA_famsa.fas has model EX_EHO+FU+R7

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E_3di_AA_famsa.fas -p refolded_fullglyco_E_3di_AA_famsa_parts.nex -B 1000


<br>

### Alignment trimming

Use trimal to make trimmed version of the famsa3di alignment and get numbers of retained columns:

`for f in E E1 E2; do trimal -in refolded_fullglyco_${f}_3di_famsa.fas -out refolded_fullglyco_${f}_3di_famsa_trim35.fas -gt 0.35 -colnumbering | tee refolded_fullglyco_${f}_3di_famsa_trim35_cols.txt; done`



In [23]:
def pass_trim(totrimal, cols, outf):
        
    newtrim = ''
    
    for rec in SeqIO.parse(totrimal, 'fasta'):
        
        trimseq = ''.join([str(rec.seq)[i] for i in range(len(rec.seq)) if i in cols])
        newtrim = newtrim + '>%s\n%s\n'%(rec.id, trimseq)
        
    with open(outf, 'w') as out:
        out.write(newtrim[:-1])


Etrim35 = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 28, 29, 30, 31, 32, 33, 44, 45, 46, 47, 48, 49, 50, 51, 52, 66, 67, 68, 69, 84, 85, 86, 87, 88, 89, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 130, 131, 132, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 162, 163, 164, 165, 166, 167, 179, 180, 181, 182, 183, 212, 213, 214, 222, 223, 224, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 247, 248, 249, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 278, 279, 280, 287, 302, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 365, 366, 367, 370, 372, 373, 374, 375, 376, 391, 392, 393, 394, 395, 396, 399, 400, 408, 409, 413, 414, 415, 416, 417, 418, 421, 422, 423, 426, 427, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 446, 447, 448, 449, 450, 451, 452, 459, 460, 461, 462, 474, 481, 482, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 520, 521, 522, 523, 524, 525, 526, 527, 531, 544, 545, 546, 547, 554, 555, 556, 567, 568, 569, 570, 571, 572, 573, 574, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 601, 602, 603, 604, 608, 609, 610, 611, 612, 613, 614, 615, 630, 631, 647, 648, 649, 650, 661, 662, 673, 674, 675, 689, 690, 691, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 708, 709, 710, 711, 712, 713, 714, 715, 716, 720, 721, 722, 723, 724, 734, 736, 737, 738, 739, 740, 748, 749, 750, 751, 752, 753, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 773, 781, 785, 786, 789, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 803, 804, 805, 806, 807, 808, 809, 810, 822, 823, 824, 825, 826, 830, 831, 832, 833, 834, 835, 838, 842, 849, 850, 851, 853, 854, 855, 856, 857, 858, 859, 860, 861, 868, 869, 870, 873, 874, 875, 876, 877, 878, 880, 888, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 924, 925, 926, 930, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 966, 967, 968, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 994, 995, 997, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019]
pass_trim(commonpath + 'E_AA_famsa.fas', Etrim35, commonpath + 'E_AA_famsa_3ditrim35.fas')

E1trim35 = [37, 38, 39, 40, 41, 42, 43, 44, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 124, 125, 126, 127, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 180, 181, 191, 192, 193, 194, 195, 196, 197, 198, 199, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 222, 223, 224, 225, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307]
pass_trim(commonpath + 'E1_AA_famsa.fas', E1trim35, commonpath + 'E1_AA_famsa_3ditrim35.fas')

E2trim35 = [56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 69, 70, 71, 72, 73, 74, 79, 80, 81, 82, 83, 84, 87, 88, 91, 92, 93, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 132, 133, 134, 135, 136, 137, 138, 154, 168, 176, 179, 180, 183, 184, 185, 186, 198, 199, 206, 207, 212, 213, 214, 215, 216, 217, 218, 233, 245, 246, 251, 252, 257, 258, 259, 262, 263, 264, 265, 266, 280, 281, 295, 296, 297, 301, 302, 303, 304, 305, 334, 335, 336, 341, 343, 344, 345, 346, 353, 354, 355, 360, 367, 368, 387, 388, 389, 390, 391, 392, 393, 394, 418, 419, 431, 432, 433, 434, 435, 460, 461, 465, 466, 467, 475, 478, 479, 480, 481, 489, 490, 491, 492, 504, 508, 509, 510, 516, 517, 518, 519, 520, 521, 522, 523, 524, 526, 537, 538, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 568, 571, 572, 573, 577, 578, 579, 580, 597, 598, 599, 600, 605, 606, 607, 608, 609, 622, 623, 634, 648, 649, 650, 651, 662, 669, 670, 671, 672, 673, 676, 686, 688, 689, 705, 706, 707, 708, 711, 712, 713, 721, 722, 723, 724, 725, 726, 728, 729, 735, 736, 737, 738, 739, 740, 745, 746, 752, 753, 754, 768, 776, 777, 779, 780, 781, 783, 789, 790, 791, 792, 793, 794, 795, 796, 805, 819, 820, 821, 822, 823, 846, 847, 848, 849, 850, 851, 852, 856, 857, 858, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 878, 879, 880, 881, 882, 883, 884, 885, 889, 890, 891, 893, 894, 895, 896, 910, 911, 912, 913, 914, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960]
pass_trim(commonpath + 'E2_AA_famsa.fas', E2trim35, commonpath + 'E2_AA_famsa_3ditrim35.fas')


<br>

Trim35 combined alignments (AA trims identical to 3di trims):

In [22]:
aa_3di_comb(commonpath + 'E1_3di_famsa_trim35.fas', commonpath + 'E1_AA_famsa_3ditrim35.fas', commonpath + 'E1_3di_t35_AA_3dit35_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E1_3di_famsa_trim35.fas has model 3DI+R5

alignment refolded_fullglyco_E1_AA_famsa_3ditrim35.fas has model EX_EHO+FU+R7

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E1_3di_t35_AA_3dit35_famsa.fas -p refolded_fullglyco_E1_3di_t35_AA_3dit35_famsa_parts.nex -B 1000


In [23]:
aa_3di_comb(commonpath + 'E2_3di_famsa_trim35.fas', commonpath + 'E2_AA_famsa_3ditrim35.fas', commonpath + 'E2_3di_t35_AA_3dit35_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E2_3di_famsa_trim35.fas has model 3DI+F+R6

alignment refolded_fullglyco_E2_AA_famsa_3ditrim35.fas has model WAG+F+R6

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E2_3di_t35_AA_3dit35_famsa.fas -p refolded_fullglyco_E2_3di_t35_AA_3dit35_famsa_parts.nex -B 1000


In [24]:
aa_3di_comb(commonpath + 'E_3di_famsa_trim35.fas', commonpath + 'E_AA_famsa_3ditrim35.fas', commonpath + 'E_3di_t35_AA_3dit35_famsa.fas', '3di/bestmodels/')

alignment refolded_fullglyco_E_3di_famsa_trim35.fas has model 3DI+F+R6

alignment refolded_fullglyco_E_AA_famsa_3ditrim35.fas has model EX_EHO+FU+R8

Run as: 
	iqtree2 -nt 22 -s refolded_fullglyco_E_3di_t35_AA_3dit35_famsa.fas -p refolded_fullglyco_E_3di_t35_AA_3dit35_famsa_parts.nex -B 1000


<br>

Finally make trees using the best substitution model for each `refolded_fullglyco_` `_3di_famsa.fas` and `_AA_famsa.fas` alignment as well as their trimmed versions 

All final tree inferences in `fn_3di_trees`

<br>

<br>

## Test the contribution of each partition to 3di+AA trees:

`iqtree2 -nt 20 -s refolded_fullglyco_E_3di_AA_famsa.fas -p refolded_fullglyco_E_3di_AA_famsa_parts.nex -z refolded_fullglyco_E_3di_AA_famsa_parts.nex.treefile -n 0 -wpl --prefix refolded_fullglyco_E_3di_AA_famsa_wpl`

repeat for E1 and E2


In [69]:
partlhrez = []
for f in [x for x in os.listdir("3di/partition_LogL/") if '.partlh' in x]:
    with open("3di/partition_LogL/" + f, 'r') as infile:
        for l in infile.readlines():
            if 'Tree' in l:
                l = l.replace('\n', '')
                l = l.split(' ')
                partlhrez.append([f.split('_wpl')[0], float(l[-2]), float(l[-1])])

partlh_df = pd.DataFrame(partlhrez)
partlh_df.columns = ['file', 'LogL_3di', 'LogL_AA']
partlh_df['LogL_diff'] = partlh_df['LogL_3di'] - partlh_df['LogL_AA']
partlh_df.to_csv('3di/partition_LogL/partition_LogL_diff.csv', index=False)

partlh_df

,file,LogL_3di,LogL_AA,LogL_diff
0,refolded_fullglyco_E1_3di_AA_famsa,-29638.0,-46320.8,16682.8
1,refolded_fullglyco_E1_3di_t35_AA_3dit35_famsa,-27824.5,-44307.7,16483.2
2,refolded_fullglyco_E2_3di_AA_famsa,-71642.1,-89190.8,17548.7
3,refolded_fullglyco_E2_3di_t35_AA_3dit35_famsa,-58443.2,-73415.9,14972.7
4,refolded_fullglyco_E_3di_AA_famsa,-99213.8,-132696.0,33482.2
5,refolded_fullglyco_E_3di_t35_AA_3dit35_famsa,-91259.1,-124080.0,32820.9


<br>

<br>

## Make clade consensus sequence alignments

In [28]:
def consensus(seqdic):
    conseq = ''
    for i in range(len(seqdic[list(seqdic)[0]])):
        thissite = []
        for rec in list(seqdic):
            thissite.append(seqdic[rec].seq[i])
    #     print(thissite)
        mostfreqsite = max(set(thissite), key = thissite.count)
        conseq = conseq + mostfreqsite
        
    return conseq

In [29]:
e1_fn_aldic = SeqIO.to_dict(SeqIO.parse('3di/refolded_fullglyco_E1_AA_famsa.fas', 'fasta'))

fasta = ''
for clad in ['HPPV', 'HPHV', 'PLPV']:
    fasta = fasta + '>%s|E1\n%s\n'%(clad, consensus({x:e1_fn_aldic[x] for x in list(e1_fn_aldic) if clad in x}))

with open('3di/consensus_al/refolded_fullglyco_E1_AA_famsa_clad_consensus.fas', 'w') as out:
    out.write(fasta[:-1])


In [30]:
e2_fn_aldic = SeqIO.to_dict(SeqIO.parse('3di/refolded_fullglyco_E2_AA_famsa.fas', 'fasta'))

fasta = ''
for clad in ['HPPV', 'HPHV', 'PLPV']:
    fasta = fasta + '>%s|E2\n%s\n'%(clad, consensus({x:e2_fn_aldic[x] for x in list(e2_fn_aldic) if clad in x}))

with open('3di/consensus_al/refolded_fullglyco_E2_AA_famsa_clad_consensus.fas', 'w') as out:
    out.write(fasta[:-1])

In [31]:
e_fn_aldic = SeqIO.to_dict(SeqIO.parse('3di/refolded_fullglyco_E_AA_famsa.fas', 'fasta'))

set([x[:4] for x in list(e_fn_aldic)])


fasta = ''
for clad in [ ['FJAF','FJFL','FJIS','FJMB','FJNV','FJTB','FJUN'], 'FJJI', 'PLLG', 'PLUN']:
    if type(clad) != list:
        fasta = fasta + '>%s|E\n%s\n'%(clad, consensus({x:e_fn_aldic[x] for x in list(e_fn_aldic) if clad in x}))
#         print(fasta)
    else:
        fasta = fasta + '>%s|E\n%s\n'%('FJortho', consensus({x:e_fn_aldic[x] for x in list(e_fn_aldic) if x[:4] in clad}))
#         print(fasta)

with open('3di/consensus_al/refolded_fullglyco_E_AA_famsa_clad_consensus.fas', 'w') as out:
    out.write(fasta[:-1])
